In [47]:
from DataIO import DataIO
from Utils import City, AggregationLevel, TrafficType
from DataPreprocessing.WeatherData.Data import IrisWeatherData
from SleepInference.SleepPreprocessor import SleepPreprocessor
from SleepInference.TimeSeriesDecomposition import Decomposer
import pandas as pd
from SleepDetection.Detector import Detector
from SleepDetection.Preprocessing import Preprocessor
from SleepDetection.Plotter import DetectionResultPlot, HistogramsSleepPatterns, UsersVsTraffic, SeasonPlot
from SleepDetection.SleepScoreEvaluation import SleepScorer
from SleepDetection.ChangePointDetection import ChangePointDetector
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_mantine_components as dmc

In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
xar_city_users = DataIO.load_traffic_data(traffic_type=TrafficType.USERS, city=City.BORDEAUX, aggregation_level=AggregationLevel.IRIS)

100%|██████████| 77/77 [00:17<00:00,  4.46it/s]


In [14]:
xar_city_traffic = DataIO.load_traffic_data(traffic_type=TrafficType.UL_AND_DL, city=City.BORDEAUX, aggregation_level=AggregationLevel.IRIS)

100%|██████████| 77/77 [00:34<00:00,  2.20it/s]


In [49]:
weather_data = IrisWeatherData()
weather_data.load_all()

(               iris  station             datetime temperature cloudiness   
 0         010010000     7481  2019-03-01 01:00:00  283.050000         90  \
 1         010010000     7481  2019-03-01 04:00:00  282.450000        100   
 2         010010000     7481  2019-03-01 07:00:00  282.650000         90   
 3         010010000     7481  2019-03-01 10:00:00  284.250000         90   
 4         010010000     7481  2019-03-01 13:00:00  284.150000         90   
 ...             ...      ...                  ...         ...        ...   
 35677183        NaN    81401  2019-05-31 23:00:00  300.250000         mq   
 35677184        NaN    81405  2019-05-31 23:00:00  300.650000         mq   
 35677185        NaN    81408  2019-05-31 23:00:00  302.750000         mq   
 35677186        NaN    81415  2019-05-31 23:00:00  304.850000         mq   
 35677187        NaN    89642  2019-05-31 23:00:00  264.950000         mq   
 
          precipitation_last_1h precipitation_last_3h precipitation_last_1

In [50]:
weather_data.weather_data.head()

,iris,station,datetime,temperature,cloudiness,precipitation_last_1h,precipitation_last_3h,precipitation_last_12h,weather_classification
0,010010000,7481,2019-03-01 01:00:00,283.050000,90,0.000000,0.000000,0.000000,3
1,010010000,7481,2019-03-01 04:00:00,282.450000,100,0.000000,0.000000,0.000000,0
2,010010000,7481,2019-03-01 07:00:00,282.650000,90,0.000000,0.000000,0.000000,0
3,010010000,7481,2019-03-01 10:00:00,284.250000,90,0.000000,0.000000,0.000000,3
4,010010000,7481,2019-03-01 13:00:00,284.150000,90,0.000000,-0.100000,-0.100000,1


In [ ]:
import numpy as np

In [44]:
new_index = np.add.outer(xar_city_users.indexes['day'], xar_city_users.indexes['time']).flatten()
b = xar_city_users.stack(datetime=['day', 'time'], create_index=False)
c = b.reindex({'datetime': new_index})

In [45]:
c.indexes

Indexes:
    iris      Index(['330030101', '330030102', '330030103', '330030104', '330040000',
       '330120000', '330130101', '330130104', '330320101', '330320102',
       ...
       '335500102', '335500103', '335500104', '335500105', '335500106',
       '335500107', '335500108', '335500109', '335500110', '335540000'],
      dtype='object', name='iris', length=305)
    service   Index(['Twitch', 'Orange_TV', 'Web_Games', 'Web_Weather', 'Twitter',
       'Apple_Music', 'Web_Ads', 'SoundCloud', 'Wikipedia', 'Web_Food',
       'YouTube', 'Pinterest', 'Web_Clothes', 'Web_Adult', 'DailyMotion',
       'Instagram', 'Clash_of_Clans', 'Pokemon_GO', 'Web_Finance',
       'Facebook_Live', 'EA_Games', 'Apple_Video', 'LinkedIn', 'Snapchat',
       'Deezer', 'Netflix', 'Facebook', 'Molotov', 'Web_e-Commerce',
       'Fortnite', 'Periscope', 'Spotify', 'Web_Streaming', 'Yahoo'],
      dtype='object', name='service')
    datetime  DatetimeIndex(['2019-03-16 00:00:00', '2019-03-16 00:15:00',
       

In [32]:
import numpy as np

In [40]:
np.add.outer(b.indexes['datetime'].levels[0], b.indexes['datetime'].levels[1]).flatten()

array(['2019-03-16T00:00:00.000000000', '2019-03-16T00:15:00.000000000',
       '2019-03-16T00:30:00.000000000', ...,
       '2019-05-31T23:15:00.000000000', '2019-05-31T23:30:00.000000000',
       '2019-05-31T23:45:00.000000000'], dtype='datetime64[ns]')

In [24]:
b.indexes['datetime'].levels[0] + b.indexes['datetime'].levels[1]

ValueError: cannot add indices of unequal length

In [11]:
SeasonPlot(xar_city=xar_city_users, n_days_season=7).plot()

In [15]:
users_vs_traffic_plot = UsersVsTraffic(xar_city_users=xar_city_users, xar_city_traffic=xar_city_traffic)
users_vs_traffic_plot.plot()

In [14]:
detector = Detector(xar_city=xar_city)
result = detector.detect_sleep_patterns()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


2023-05-30 13:46:34,674 - SleepInferenceLogger - WARNING - Segmentation [ 0 30 40 60 61 96] has a segment that is too small to compute the cost. Cost is set to nan.
2023-05-30 13:46:34,674 - SleepInferenceLogger - WARNING - Segmentation [ 0 30 40 60 62 96] has a segment that is too small to compute the cost. Cost is set to nan.
2023-05-30 13:46:34,683 - SleepInferenceLogger - WARNING - Segmentation [ 0 29 44 61 62 96] has a segment that is too small to compute the cost. Cost is set to nan.
2023-05-30 13:46:34,683 - SleepInferenceLogger - WARNING - Segmentation [ 0 29 44 61 63 96] has a segment that is too small to compute the cost. Cost is set to nan.


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


2023-05-30 13:46:35,982 - SleepInferenceLogger - WARNING - Segmentation [ 0 30 40 60 61 96] has a segment that is too small to compute the cost. Cost is set to nan.
2023-05-30 13:46:35,982 - SleepInferenceLogger - WARNING - Segmentation [ 0 30 40 60 62 96] has a segment that is too small to compute the cost. Cost is set to nan.
2023-05-30 13:46:36,003 - SleepInferenceLogger - WARNING - Segmentation [ 0 29 44 61 62 96] has a segment that is too small to compute the cost. Cost is set to nan.
2023-05-30 13:46:36,003 - SleepInferenceLogger - WARNING - Segmentation [ 0 29 44 61 63 96] has a segment that is too small to compute the cost. Cost is set to nan.


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


2023-05-30 13:47:01,215 - SleepInferenceLogger - WARNING - Segmentation [ 0 30 46 60 62 96] has a segment that is too small to compute the cost. Cost is set to nan.
2023-05-30 13:47:01,258 - SleepInferenceLogger - WARNING - Segmentation [ 0 32 48 61 63 96] has a segment that is too small to compute the cost. Cost is set to nan.


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


2023-05-30 13:47:01,549 - SleepInferenceLogger - WARNING - Segmentation [ 0 30 46 60 62 96] has a segment that is too small to compute the cost. Cost is set to nan.
2023-05-30 13:47:01,585 - SleepInferenceLogger - WARNING - Segmentation [ 0 32 48 61 63 96] has a segment that is too small to compute the cost. Cost is set to nan.


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished


In [20]:
plotter = DetectionResultPlot(detection_result=result)
figures = plotter.plot()

In [29]:
external_stylesheets = [dmc.theme.DEFAULT_COLORS]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(
    style={'overflowY': 'scroll', 'height': '1200px'},  # This enables scrolling
    children=[dcc.Graph(figure=fig) for fig in figures]
)

In [30]:
app.run_server(mode='external')  # Turn off reloader if inside Jupyter

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


In [245]:
histograms = HistogramsSleepPatterns(sleep_change_points=result.sleep_change_points)
figures = histograms.plot()

In [25]:
loc_ids = [c for c in result.sleep_change_points.columns if not c.startswith('unc_')]
traffic = result.traffic_time_series_data[loc_ids]
daily = result.daily_component_traffic_time_series_data[loc_ids]
sleep_cp = result.sleep_change_points
sleep_sc = result.sleep_scores

In [28]:
base = '/Users/anmusso/Desktop/PhD/NetMob/SleepDetection'
traffic.to_csv(f'{base}/Temp/DataVizConsulting/traffic_data.csv')
daily.to_csv(f'{base}/Temp/DataVizConsulting/daily_component_traffic.csv')
sleep_cp.to_csv(f'{base}/Temp/DataVizConsulting/sleep_change_points.csv')
sleep_sc.to_csv(f'{base}/Temp/DataVizConsulting/sleep_score.csv')